In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio

In [3]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


#tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=limit, n_frames=600, version='dev-clean')

train_dataloader = data.DataLoader(training_set, **params)
dev_dataloader = data.DataLoader(dev_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

#vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

CPU times: user 325 ms, sys: 77.3 ms, total: 403 ms
Wall time: 1.08 s


In [5]:
#tknz = modern_tokenizers()

LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')[0]

(tensor([[[-1.4131, -1.2465, -1.0977,  ...,  0.0000,  0.0000,  0.0000],
          [-0.6647, -0.7953, -0.8671,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0141, -0.2716, -0.4297,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.1281, -0.2637, -0.4295,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0137,  0.3066, -0.0938,  ...,  0.0000,  0.0000,  0.0000],
          [-0.7350, -0.3410, -0.2404,  ...,  0.0000,  0.0000,  0.0000]]]),
 tensor([  101,   101,  2045,  2001,  2625,  1997,  1037,  4306,  2055,  2085,
          1998, 14163,  6894,  5289,  2106,  2025,  2031,  2000,  2562,  2067,
          1037,  5481,  2004,  2002,  2441,  1996,  9445,  3460,  6031,  2988,
          1996, 14460,  4214,  8721,  2000, 10767,  1998, 27468,  1996,  2221]))

In [ ]:
# 'bahdanau_basic'
# 'bahdanau_audio'
# 'luong_dot'
# 'luong_concat'
# 'luong_general'
# 'super_attn'

In [6]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)

decoder = DecoderATTRNN(30000, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64).to(device)


encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [8]:
# Train the model
global_trainer(10, train_dataloader, dev_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'],  bad_audio_mode=True)

 ====== ====== ====== ====== ====== ======
      The model has 4483691 parameters
 ====== ====== ====== ====== ====== ======


Epoch       10: 100%|████████████████████| 4/4 [01:04<00:00, 16.17s/it, Train loss 5.8535 Eval loss 6.1365]

Time taken for the training 0.18373 hours



In [ ]:
mfccs, references = training_set[1]
references = [tokenizer.index_word[ind] for ind in references.numpy()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang=tokenizer, 
          device=device, beam_search=True)

In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU

In [ ]:

# Faire un échéancier conforme à un des papiers
# Vérifier pourquoi les sorties de spec_augment donne des nan
# Revoir l'Evaluation 

